In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [5]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd

In [7]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [8]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [9]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0730').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,31.04,2025-12-23,2026-01-20,30.05,36.03,33.74,0.071950
9,IXN,105.34,2025-12-23,2026-01-20,104.16,115.14,113.59,0.053383
17,SMH,362.47,2025-12-23,2026-01-20,355.83,393.60,392.16,0.049825
26,XLK,145.56,2025-12-23,2026-01-20,143.42,159.97,154.21,0.047907
16,QQQ,621.21,2025-12-23,2026-01-20,612.43,682.21,658.11,0.047821
29,XLV,155.15,2025-12-23,2026-01-20,147.44,158.38,139.79,-0.042625
32,XSW,190.69,2025-12-23,2026-01-20,189.20,205.61,197.64,0.035625
12,IWO,332.85,2025-12-23,2026-01-20,330.90,359.38,342.24,0.034019
10,IXP,120.68,2025-12-23,2026-01-20,119.84,127.42,127.07,0.033947
0,ARKF,48.97,2025-12-23,2026-01-20,48.58,52.15,50.89,0.032060


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
dfp = run_model_nn_cmd('pennystock', '1226')

In [ ]:
dfp[dfp['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
39,INVZ,0.95,2025-12-26,2026-01-23,0.90,1.17,1.21,0.150877
38,INLX,8.14,2025-12-26,2026-01-23,7.51,10.46,10.12,0.150287
85,ZYXI,0.13,2025-12-24,2026-01-21,0.11,0.16,0.17,0.128205
27,EVGO,3.14,2025-12-26,2026-01-23,3.00,4.13,3.48,0.126327
37,INDI,3.74,2025-12-26,2026-01-23,3.53,4.72,4.21,0.110517
81,VTEX,3.71,2025-12-26,2026-01-23,3.55,4.81,3.99,0.109614
75,SST,4.61,2025-12-26,2026-01-23,4.38,6.02,4.88,0.104845
11,BGSF,4.76,2025-12-26,2026-01-23,4.43,6.06,4.99,0.084034
5,AUR,4.00,2025-12-26,2026-01-23,3.72,4.87,4.35,0.078333
33,GTE,4.15,2025-12-26,2026-01-23,3.54,5.92,3.94,0.076305


In [10]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
1,ACN,268.30,2025-12-31,2026-01-28,260.91,285.82,310.67,0.065225
2,ADBE,349.99,2025-12-31,2026-01-28,339.55,405.16,370.05,0.061707
19,FDX,288.86,2025-12-31,2026-01-28,248.57,289.82,277.57,-0.058414
33,PYPL,58.38,2025-12-31,2026-01-28,58.10,64.02,62.54,0.054357
24,KO,69.91,2025-12-31,2026-01-28,62.28,71.08,66.03,-0.049301
21,INTC,36.90,2025-12-31,2026-01-28,32.63,38.47,34.38,-0.047154
37,TSLA,449.72,2025-12-31,2026-01-28,397.58,459.59,430.26,-0.045754
39,UNH,330.11,2025-12-31,2026-01-28,323.55,355.71,351.85,0.041178
15,DELL,125.88,2025-12-31,2026-01-28,115.40,129.96,119.07,-0.034980
3,AMD,214.16,2025-12-31,2026-01-28,208.95,240.01,212.15,0.028997


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
24,XCEM,38.65,2025-12-11,2026-01-08,37.00,39.03,36.46,-0.029840
12,EWT,65.96,2025-12-11,2026-01-08,63.24,66.55,63.47,-0.023347
4,EPHE,25.00,2025-12-11,2026-01-08,24.15,27.49,24.85,0.019867
1,EWD,49.65,2025-12-11,2026-01-08,47.89,50.19,48.51,-0.015844
7,EWI,53.97,2025-12-11,2026-01-08,51.13,55.02,53.24,-0.015564
13,EWW,71.21,2025-12-11,2026-01-08,67.04,72.61,70.98,-0.014043
21,THD,60.00,2025-12-11,2026-01-08,57.67,61.71,58.28,-0.013000
23,VNM,18.28,2025-12-11,2026-01-08,17.65,18.65,17.90,-0.011670
18,INDA,53.62,2025-12-11,2026-01-08,51.86,55.47,55.35,0.011314
8,EWJ,84.29,2025-12-11,2026-01-08,82.69,86.88,80.53,-0.010954


In [ ]:
run_model_nn_cmd('global', '1107').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
45,NVO,49.18,2025-12-10,2026-01-07,46.46,59.29,52.99,0.075912
32,IFNNy,42.80,2025-12-10,2026-01-07,42.53,47.23,48.31,0.075312
61,TGLS,52.88,2025-12-10,2026-01-07,51.80,61.16,57.32,0.073374
23,ESEA,59.10,2025-12-10,2026-01-07,58.40,67.25,64.57,0.072871
1,ADDDF,184.14,2025-12-09,2026-01-06,179.08,215.78,193.40,0.064878
47,ONON,49.02,2025-12-10,2026-01-07,47.80,56.93,51.35,0.061336
11,CDMGF,23.80,2025-12-09,2026-01-06,21.04,23.90,22.47,-0.055882
31,HENKY,18.80,2025-12-10,2026-01-07,16.27,19.53,17.52,-0.054610
3,BAK,2.96,2025-12-10,2026-01-07,2.72,3.28,3.36,0.054054
35,MELI,1979.51,2025-12-10,2026-01-07,1883.24,2220.34,2095.45,0.043866


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
7,ARE,45.56,2025-12-11,2026-01-08,47.62,52.79,48.85,0.092040
83,SWK,76.32,2025-12-11,2026-01-08,75.55,86.74,84.73,0.078878
27,COP,96.61,2025-12-11,2026-01-08,94.51,107.14,107.27,0.065866
70,POR,47.49,2025-12-11,2026-01-08,41.86,48.08,43.19,-0.065558
48,HPE,24.45,2025-12-11,2026-01-08,24.33,27.06,26.53,0.062304
95,WHR,78.75,2025-12-11,2026-01-08,78.79,84.22,87.45,0.060148
36,ENTG,95.47,2025-12-11,2026-01-08,91.60,106.59,104.32,0.056213
41,FLO,10.86,2025-12-11,2026-01-08,10.93,11.77,11.61,0.053100
75,ROP,445.74,2025-12-11,2026-01-08,439.13,482.67,483.15,0.050650
50,IP,39.37,2025-12-11,2026-01-08,36.82,43.47,43.61,0.049022


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
9,SOYB,22.52,2025-12-12,2026-01-09,21.94,23.66,22.72,0.011249
15,WEAT,20.52,2025-12-12,2026-01-09,20.05,21.13,20.76,0.006173
4,FXY,58.99,2025-12-12,2026-01-09,57.52,61.00,59.54,0.006159
12,USO,68.81,2025-12-12,2026-01-09,67.15,70.91,69.54,0.005668
1,FXB,128.40,2025-12-12,2026-01-09,125.18,132.64,129.54,0.005607


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
11,ICP-USD,3.29,2025-12-12,2026-01-09,3.23,4.43,3.81,0.162107
18,ATOM-USD,2.14,2025-12-12,2026-01-09,1.97,2.71,2.32,0.090343
3,ADA-USD,0.41,2025-12-12,2026-01-09,0.40,0.50,0.44,0.089431
0,BTC-USD,90229.87,2025-12-12,2026-01-09,86585.86,108093.01,95716.62,0.072799
12,AVAX-USD,13.10,2025-12-12,2026-01-09,12.33,16.13,13.26,0.061578


In [ ]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
53,OKTA,89.64,2025-12-19,2026-01-16,88.92,108.35,99.79,0.104641
40,KD,27.17,2025-12-19,2026-01-16,26.76,29.12,33.87,0.101092
76,U,45.86,2025-12-19,2026-01-16,39.13,45.76,39.17,-0.098270
8,BYRN,17.40,2025-12-19,2026-01-16,17.28,20.35,19.67,0.097701
48,MU,263.83,2025-12-19,2026-01-16,262.50,301.18,305.01,0.097538
64,SMCI,30.36,2025-12-19,2026-01-16,30.17,36.45,32.93,0.092995
83,ZS,231.10,2025-12-19,2026-01-16,236.88,271.54,248.95,0.092413
42,MBLY,10.24,2025-12-19,2026-01-16,10.43,11.51,11.53,0.089518
22,ELF,79.65,2025-12-19,2026-01-16,78.18,97.31,83.98,0.085876
23,ENPH,33.41,2025-12-19,2026-01-16,32.56,38.70,37.57,0.085803


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [11]:
import sys
sys.path.append('/content')
# !unzip drive/MyDrive/colab/stock_prediction_r.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [ ]:
run_model_nn_cmd('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1220'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_close_model.pth'}


In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
26,EXPE,289.25,2025-12-19,2026-01-16,189.62,291.44,189.38,-0.227381
48,MU,265.92,2025-12-19,2026-01-16,174.66,270.49,171.69,-0.226785
11,COHR,185.83,2025-12-19,2026-01-16,108.07,191.14,134.34,-0.222318
5,APP,721.37,2025-12-19,2026-01-16,553.35,728.21,491.45,-0.180721
77,UI,564.62,2025-12-19,2026-01-16,388.92,595.41,428.44,-0.165946
51,NVCR,12.84,2025-12-19,2026-01-16,11.25,16.00,16.78,0.143043
73,TRIP,13.87,2025-12-19,2026-01-16,12.95,16.06,18.45,0.140591
33,HOOD,121.35,2025-12-19,2026-01-16,92.11,122.82,99.67,-0.135833
42,MBLY,10.19,2025-12-19,2026-01-16,9.87,12.39,11.95,0.119071
72,TOST,36.22,2025-12-19,2026-01-16,29.83,36.82,30.22,-0.108504


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
defetfcfg.model_path = f'/{colab_path}/models/rolling/etf_1221'
etfdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
etfdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/etf_1221_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/etf_1221_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/etf_1221_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,31.12,2025-12-23,2026-01-20,30.03,33.07,34.27,0.042952
17,SMH,362.57,2025-12-23,2026-01-20,350.09,383.28,387.92,0.030872
32,XSW,190.69,2025-12-23,2026-01-20,183.57,201.97,203.57,0.029787
26,XLK,145.59,2025-12-23,2026-01-20,140.50,153.66,155.37,0.029214
0,ARKF,48.98,2025-12-23,2026-01-20,47.21,51.84,52.02,0.028107
2,ARKW,154.10,2025-12-23,2026-01-20,148.54,162.98,163.73,0.028012
16,QQQ,621.32,2025-12-23,2026-01-20,599.29,655.97,660.43,0.027753
18,SPY,687.58,2025-12-23,2026-01-20,663.48,725.22,728.49,0.026397
7,IPO,46.94,2025-12-23,2026-01-20,45.28,49.61,49.59,0.025991
5,EEM,54.24,2025-12-23,2026-01-20,52.32,57.21,57.41,0.025934


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1226'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_close_model.pth'}


In [ ]:
tvoldf[tvoldf['predicted_return']> 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
6,ATOM,2.18,2025-12-26,2026-01-23,2.14,3.61,2.33,0.235474
67,YEXT,8.05,2025-12-26,2026-01-23,7.86,11.77,8.65,0.171014
58,SLE,0.62,2025-12-26,2026-01-23,0.52,0.91,0.74,0.166667
31,IRDM,17.17,2025-12-26,2026-01-23,16.32,20.08,23.59,0.164628
63,TDOC,7.18,2025-12-26,2026-01-23,6.49,10.70,7.46,0.144383
40,NNOX,2.85,2025-12-26,2026-01-23,2.66,4.12,3.00,0.143860
61,STEM,16.19,2025-12-26,2026-01-23,13.66,28.20,13.48,0.139386
49,PLUG,2.10,2025-12-26,2026-01-23,1.86,3.07,2.22,0.134921
30,INVZ,0.95,2025-12-26,2026-01-23,0.90,1.34,0.99,0.133333
54,RVYL,0.18,2025-12-26,2026-01-23,0.16,0.28,0.17,0.129630


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,AMD,217.53,2025-11-28,2025-12-26,199.40,287.08,317.09,0.231355
37,TSLA,430.17,2025-11-28,2025-12-26,320.48,472.67,344.03,-0.118813
21,INTC,40.56,2025-11-28,2025-12-26,31.13,42.14,37.42,-0.090319
15,DELL,133.35,2025-11-28,2025-12-26,106.26,143.03,123.80,-0.067392
41,WMT,110.51,2025-11-28,2025-12-26,94.59,112.03,102.90,-0.066389
9,CAT,575.76,2025-11-28,2025-12-26,480.11,601.42,539.90,-0.061281
38,TXN,168.27,2025-11-28,2025-12-26,154.94,173.72,145.51,-0.060696
39,UNH,329.77,2025-11-28,2025-12-26,280.70,341.08,309.39,-0.058768
2,ADBE,320.13,2025-11-28,2025-12-26,312.50,347.13,346.57,0.047699
22,IBM,308.58,2025-11-28,2025-12-26,268.36,323.14,291.86,-0.045780


In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_1208'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1208_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,AMZN,226.97,2025-12-08,2026-01-05,198.64,236.50,214.70,-0.045630
6,BA,205.96,2025-12-08,2026-01-05,189.70,211.89,242.01,0.041626
36,TMO,568.35,2025-12-08,2026-01-05,561.39,623.22,588.24,0.039764
16,DHR,224.22,2025-12-08,2026-01-05,220.64,237.45,240.39,0.038385
34,QCOM,174.03,2025-12-08,2026-01-05,156.34,177.94,168.55,-0.036890
33,PYPL,61.63,2025-12-08,2026-01-05,59.71,66.11,65.83,0.036562
2,ADBE,339.86,2025-12-08,2026-01-05,330.80,366.09,358.46,0.035083
35,SBUX,83.74,2025-12-08,2026-01-05,81.79,91.63,85.50,0.030650
8,BRK-B,495.77,2025-12-08,2026-01-05,482.76,520.51,523.54,0.026558
28,MSFT,489.86,2025-12-08,2026-01-05,465.96,509.54,455.55,-0.026218


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_1115'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [ ]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
130,LFMD,4.63,2025-11-14,2026-10-30,3.63,11.99,64.18,4.745140
356,LAES,4.49,2025-11-14,2026-10-30,3.21,39.63,30.40,4.437268
327,OKLO,97.57,2025-11-14,2026-10-30,82.54,984.50,406.78,4.035086
315,INOD,57.21,2025-11-14,2026-10-30,39.89,373.78,401.43,3.749170
345,NUTX,107.11,2025-11-14,2026-10-30,100.94,336.41,954.96,3.332960
95,CHGG,1.09,2025-11-14,2026-10-30,0.58,4.85,8.61,3.293578
329,RGTI,25.48,2025-11-14,2026-10-30,19.92,215.40,82.17,3.153454
330,ACHR,7.88,2025-11-14,2026-10-30,7.71,64.69,23.58,3.060068
258,WKEY,9.20,2025-11-14,2026-10-30,7.15,37.14,65.95,2.994203
352,BULL,8.63,2025-11-14,2026-10-30,4.07,57.04,40.67,2.931248


In [ ]:
buystkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
buystkrcfg.model_path = f'/{colab_path}/models/rolling/buystocks_1213'
buystkrcfg.model.window_size = 500
buystkrcfg.model.future_days = 250
buystkrdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkrcfg)

In [ ]:
buystkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
159,LAZR,0.90,2025-12-12,2026-11-27,0.78,4.36,2.42,1.800000
160,NOW,865.06,2025-12-12,2026-11-27,743.05,3287.56,1157.09,0.998975
358,SQNS,5.95,2025-12-12,2026-11-27,5.34,19.88,9.82,0.963025
246,COSM,0.44,2025-12-12,2026-11-27,0.40,1.05,0.78,0.689394
5,DOMO,9.04,2025-12-12,2026-11-27,8.56,15.78,21.07,0.674410
35,UP,0.70,2025-12-12,2026-11-27,0.61,1.44,1.45,0.666667
349,BULL,9.16,2025-12-12,2026-11-27,8.34,26.16,9.34,0.595342
23,MBLY,11.07,2025-12-12,2026-11-27,9.79,23.06,19.97,0.590485
130,LFMD,3.40,2025-12-12,2026-11-27,3.35,6.58,6.11,0.572549
366,QUBT,12.05,2025-12-12,2026-11-27,8.94,22.31,24.24,0.534993


In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_1012'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

In [ ]:
stkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
111,LUMN,8.69,2025-12-10,2026-01-07,6.32,12.09,14.69,0.269659
368,LAES,4.77,2025-12-10,2026-01-07,4.01,8.23,5.40,0.232704
250,DXYZ,31.85,2025-12-10,2026-01-07,29.49,43.58,40.41,0.187650
342,AEVA,15.44,2025-12-10,2026-01-07,13.92,17.00,23.78,0.180915
358,BMNR,40.40,2025-12-10,2026-01-07,31.85,41.67,25.77,-0.180776
32,DAPP,19.87,2025-12-10,2026-01-07,18.26,24.33,27.54,0.176480
355,NUTX,176.47,2025-12-10,2026-01-07,155.69,245.25,220.23,0.173325
102,AEHR,25.84,2025-12-10,2026-01-07,23.61,37.54,29.02,0.163184
46,RMBS,106.84,2025-12-10,2026-01-07,89.61,124.57,158.47,0.162642
369,OPEN,7.00,2025-12-10,2026-01-07,4.63,7.38,5.67,-0.158095


In [19]:
stkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
stkrcfg.model_path = f'/{colab_path}/models/rolling/stock_0102'
stkrdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/stock_0102_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/stock_0102_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/stock_0102_close_model.pth'}


In [20]:
stkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
258,EOSE,11.46,2025-12-31,2026-01-28,8.43,14.79,16.00,0.140780
372,SQNS,4.49,2025-12-31,2026-01-28,4.17,5.27,5.85,0.135115
95,ORCL,194.91,2025-12-31,2026-01-28,173.95,274.66,203.55,0.115318
354,NUTX,164.62,2025-12-31,2026-01-28,129.00,172.45,138.69,-0.108776
312,SERV,10.38,2025-12-31,2026-01-28,8.38,12.83,13.30,0.108221
218,LEU,242.76,2025-12-31,2026-01-28,226.17,288.16,288.03,0.101719
70,TEM,59.05,2025-12-31,2026-01-28,56.00,65.84,71.57,0.091787
381,QUBT,10.26,2025-12-31,2026-01-28,7.65,11.10,9.22,-0.091293
264,LUNR,16.23,2025-12-31,2026-01-28,15.28,20.21,17.61,0.090573
326,PONY,14.50,2025-12-31,2026-01-28,13.37,18.38,15.61,0.088736


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0907'
glbdf = run_model_nn(f'globaltickers.csv', cfg=glbcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/global_0907_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/global_0907_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/global_0907_close_model.pth'}


In [ ]:
glbdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
33,HMC,29.73,2025-11-07,2025-12-05,29.62,34.22,34.07,0.097769
3,BAK,2.44,2025-11-07,2025-12-05,2.45,2.77,2.75,0.088798
38,LIN,422.22,2025-11-07,2025-12-05,420.74,476.39,477.08,0.084908
56,SE,146.78,2025-11-07,2025-12-05,141.42,163.70,169.18,0.077122
66,VNT,37.79,2025-11-07,2025-12-05,37.83,41.91,42.27,0.076211
50,POAHF,40.37,2025-11-06,2025-12-04,40.64,44.84,44.72,0.075056
59,TEF,4.19,2025-11-07,2025-12-05,4.19,4.57,4.74,0.073986
46,NXPI,199.93,2025-11-07,2025-12-05,195.62,219.84,228.47,0.073592
45,NVO,45.17,2025-11-07,2025-12-05,43.24,50.50,50.34,0.063243
35,MELI,2090.31,2025-11-07,2025-12-05,2077.38,2341.55,2241.28,0.062077


In [ ]:
glbrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
glbrcfg.model_path = f'/{colab_path}/models/rolling/global_1210'
glbrdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/global_1210_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/global_1210_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/global_1210_close_model.pth'}


In [ ]:
glbrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
34,KNNGF,66.45,2025-10-23,2025-11-20,54.51,66.45,58.78,-0.098370
57,SE,124.22,2025-12-10,2026-01-07,120.04,137.44,150.25,0.094107
37,RACE,364.04,2025-12-10,2026-01-07,364.08,407.93,406.34,0.078957
51,PROSF,60.00,2025-12-08,2026-01-05,56.53,69.72,66.23,0.069333
24,EVKIY,7.54,2025-12-09,2026-01-06,7.39,8.82,7.57,0.051282
35,MELI,1970.73,2025-12-10,2026-01-07,1956.95,2137.48,2095.23,0.046932
23,ESEA,59.00,2025-12-10,2026-01-07,56.62,63.81,64.52,0.044915
56,SAP,245.88,2025-12-10,2026-01-07,236.16,263.98,269.01,0.042717
61,TGLS,53.41,2025-12-10,2026-01-07,45.93,59.29,48.65,-0.039693
18,ENGGY,7.96,2025-12-10,2026-01-07,7.70,8.42,8.61,0.035595


In [ ]:
ctcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
ctcfg.model_path = f'/{colab_path}/models/transformer/country_1211'
ctdf = run_model_nn(f'/{colab_path}/tickers/countrytickers.csv', cfg=ctcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/country_1211_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/country_1211_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/country_1211_close_model.pth'}


In [ ]:
ctdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,39.48,2025-12-11,2026-01-08,33.26,39.97,35.66,-0.080632
0,ARGT,91.19,2025-12-11,2026-01-08,90.88,101.31,100.06,0.068282
11,EWP,53.54,2025-12-11,2026-01-08,49.38,54.03,49.98,-0.045013
5,EWA,26.71,2025-12-11,2026-01-08,25.28,26.69,24.93,-0.040309
1,EWD,49.70,2025-12-11,2026-01-08,47.42,49.82,46.30,-0.037290
16,IDX,17.23,2025-12-11,2026-01-08,15.67,17.30,16.81,-0.036951
6,EWG,42.46,2025-12-11,2026-01-08,39.70,42.48,40.86,-0.034071
14,EWY,95.07,2025-12-11,2026-01-08,84.34,94.61,96.79,-0.033204
13,EWW,71.42,2025-12-11,2026-01-08,70.08,71.49,68.28,-0.020582
2,FLN,22.91,2025-12-11,2026-01-08,21.06,23.02,23.28,-0.019933


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
valrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
valrcfg.model_path = f'/{colab_path}/models/rolling/valuestock_1211'
valrdf = run_model_nn(f'/{colab_path}/tickers/valuestocktickers.csv', cfg=valrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/valuestock_1211_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/valuestock_1211_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/valuestock_1211_close_model.pth'}


In [ ]:
valrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
16,CC,13.19,2025-12-11,2026-01-08,10.59,13.79,11.58,-0.091231
94,WDC,187.62,2025-12-11,2026-01-08,166.83,195.25,159.78,-0.072842
58,MOD,164.74,2025-12-11,2026-01-08,133.77,173.57,161.98,-0.050382
8,ASC,11.33,2025-12-11,2026-01-08,10.15,11.93,10.22,-0.049721
5,APO,150.82,2025-12-11,2026-01-08,126.84,154.73,148.48,-0.049529
74,RH,154.88,2025-12-11,2026-01-08,137.08,178.85,170.72,0.047370
18,CCJ,95.83,2025-12-11,2026-01-08,87.67,108.55,104.08,0.044558
35,EQT,56.14,2025-12-11,2026-01-08,53.12,62.41,60.36,0.044353
36,ENTG,95.49,2025-12-11,2026-01-08,89.55,105.27,104.05,0.043286
53,KD,27.27,2025-12-11,2026-01-08,24.80,28.43,25.14,-0.042049


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_1212'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_1212_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_1212_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_1212_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,56.10,2025-12-12,2026-01-09,50.85,57.68,54.58,-0.030838
11,TLT,87.33,2025-12-12,2026-01-09,86.86,91.14,88.51,0.017253
1,FXB,128.40,2025-12-12,2026-01-09,127.80,132.39,130.81,0.015057
7,PALL,137.00,2025-12-12,2026-01-09,125.52,148.57,131.23,-0.013820
15,WEAT,20.52,2025-12-12,2026-01-09,19.93,20.76,20.19,-0.011046
2,FXE,108.29,2025-12-12,2026-01-09,106.98,108.95,106.24,-0.008311
10,TIP,110.21,2025-12-12,2026-01-09,110.10,111.41,111.86,0.008287
5,GLD,395.17,2025-12-12,2026-01-09,374.13,411.72,390.12,-0.008047
4,FXY,58.98,2025-12-12,2026-01-09,57.28,59.61,58.72,-0.007517
13,UUP,27.95,2025-12-12,2026-01-09,27.33,28.23,27.67,-0.007394


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,XLM-USD,0.23,2025-11-21,2025-12-19,0.22,0.28,0.29,0.144928
6,DOGE-USD,0.14,2025-11-21,2025-12-19,0.14,0.16,0.18,0.142857
3,ADA-USD,0.41,2025-11-21,2025-12-19,0.39,0.48,0.53,0.138211
4,XRP-USD,1.96,2025-11-21,2025-12-19,1.93,2.34,2.39,0.132653
5,DOT-USD,2.37,2025-11-21,2025-12-19,2.25,3.10,2.64,0.123769
11,ICP-USD,4.36,2025-11-21,2025-12-19,3.64,4.84,5.74,0.087156
12,AVAX-USD,13.32,2025-11-21,2025-12-19,8.87,16.84,11.20,-0.076326
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
18,ATOM-USD,2.53,2025-11-21,2025-12-19,2.31,3.29,2.50,0.067194
16,AAVE-USD,157.80,2025-11-21,2025-12-19,152.88,185.84,162.92,0.059654


In [ ]:
cryprcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
cryprcfg.model_path = f'/{colab_path}/models/rolling/crypto_1212'
cryprdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryprcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/crypto_1212_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/crypto_1212_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/crypto_1212_close_model.pth'}


In [ ]:
cryprdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
10,BCH-USD,578.95,2025-12-12,2026-01-09,489.04,614.08,495.46,-0.079610
0,BTC-USD,90225.23,2025-12-12,2026-01-09,84226.50,100790.43,106996.04,0.078830
18,ATOM-USD,2.14,2025-12-12,2026-01-09,1.94,2.51,2.43,0.071651
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.18,0.24,0.20,-0.060606
6,DOGE-USD,0.14,2025-12-12,2026-01-09,0.12,0.15,0.13,-0.047619
7,FIL-USD,1.32,2025-12-12,2026-01-09,1.25,1.51,1.37,0.042929
5,DOT-USD,2.00,2025-12-12,2026-01-09,1.91,2.24,2.05,0.033333
2,SOL-USD,131.90,2025-12-12,2026-01-09,114.38,149.90,119.42,-0.030326
3,ADA-USD,0.41,2025-12-12,2026-01-09,0.38,0.48,0.40,0.024390
12,AVAX-USD,13.09,2025-12-12,2026-01-09,10.60,14.69,13.14,-0.021390


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_1206'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_1206_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_1206_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_1206_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,URA,47.92,2025-12-05,2026-01-02,35.60,48.43,37.91,-0.151781
2,ARKW,155.82,2025-12-05,2026-01-02,130.61,163.09,130.07,-0.093463
0,ARKF,49.99,2025-12-05,2026-01-02,41.68,50.90,44.77,-0.084150
1,ARKK,81.91,2025-12-05,2026-01-02,67.07,87.07,71.94,-0.079966
17,SMH,364.42,2025-12-05,2026-01-02,299.18,378.65,329.92,-0.078216
7,IPO,47.59,2025-12-05,2026-01-02,41.49,49.11,43.71,-0.059256
9,IXN,106.51,2025-12-05,2026-01-02,92.00,109.05,100.04,-0.057710
32,XSW,191.70,2025-12-05,2026-01-02,170.91,195.70,178.99,-0.051295
27,XLP,78.46,2025-12-05,2026-01-02,78.07,84.04,82.45,0.039001
31,XME,99.74,2025-12-05,2026-01-02,92.36,103.15,92.30,-0.038132


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv(f'/{colab_path}/tickers/stocktickers.csv', index=False)
df.to_csv(f'/{colab_path}/tickers/buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
